# Сентимент-анализ отзывов на товары

**Соревнования Kaggle https://www.kaggle.com/c/product-reviews-sentiment-analysis **

К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется.

Поскольку тестовая выборка маленькая, на ней обучать модель мы не можем, поэтому необходимо собрать отзывы на подобные товары с разметкой, и обучить модель на них. Сначала необходимо собрать отзывы для тренировки модели, которая будет классифицировать отзывы заказчика.  

План работ будет следующий:   
**1)** С помощью парсера собирать достаточное количество отзывов на подобные товары для тренировки модели.  
**2)** Предварительно подготавить данные  
**3)** Выбрать и обучить модель на собранных отзывах.  
**4)** Сделать предсказание на тестовых данных  

## 1) Написание парсера. Сбор тренировочных данных  

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import csv  

In [2]:
test_file = 'D:\\Python\\Kaggle\\Sent_an\\test.csv'
with open(test_file,encoding="utf-8") as f:
    test = f.read()
    
test_page = BeautifulSoup(test, 'lxml')
test_reviews = test_page.findAll('review')

test_reviews_list = []
for i in range(len(test_reviews)):
    test_reviews_list += [test_reviews[i].text.replace('\n',' ').strip()]
    
test_reviews_list[:3]

['Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти. Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.',
 'ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если  постоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.  Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычных сенсорниках и кноп

Из полученного списка видно, что это отзывы на мобильные телефоны. Соберем тренировочную выборку.
Для этого будем парсить сайт https://torg.mail.ru/review/goods/mobilephones  
Анализируем первую страницу с отзывами и определяем в каких блоках находится сами отзывы, а в каких оценки.

In [3]:
url = 'https://torg.mail.ru/review/goods/mobilephones/'
req = requests.get(url)
page =BeautifulSoup(req.text, 'lxml')
# print(page.prettify()) - позволяет просмотреть код страницы

In [4]:
texts = []
rating = []
for i_page in range(0, 400):
    url = 'https://torg.mail.ru/review/goods/mobilephones/?page=' + str(i_page)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    rev = soup.find('section', class_= "card__responses js-review_list js-ustat_container js-ustat_container_reviewsList").find_all('div', class_='review-item__body')
    for r in rev:
        review = r.find('span', class_='js-more-text').text.strip()
        mark = r.find('span', class_='review-item__rating-counter').text.strip().replace(",",".")
        texts.append(review)
        rating.append(float(mark))       
      

In [5]:
print(len(texts))
print(len(rating))

8000
8000


Как и ожидалось количество отзывов 8000.  

Сохраним данные в csv файл, для последующего использования 

In [6]:
mail_reviews = pd.DataFrame()
mail_reviews['text'] = texts
mail_reviews['rating'] = rating
mail_reviews.to_csv(r'D:\Python\Kaggle\Sent_an\mail_reviews.csv', encoding='utf-8', sep=',')

In [3]:
data = pd.read_csv(r'D:\Python\Kaggle\Sent_an\mail_reviews.csv', sep=',')
texts = data['text'].tolist()
rating = data['rating'].tolist()

Отзывы с оценкой 4.5 и выше примем за положительные 'pos', остальные негативные 'neg'

In [4]:
rating2 = []
for r in rating:
    if  r >= 4.5:
        rating2.append('pos')
    else:
        rating2.append('neg')


Подсчитаем сколько положительных и отрицательных отзывов в полученной выборке, для этого воспользуемся встроенным классом Counter из модуля collections:

In [5]:
from collections import Counter
c = Counter(rating2)
c

Counter({'neg': 3415, 'pos': 4585})

Видим, что полученная выборка несбалансированная, это следует учитывать при построении модели.

# 2. Предварительная подготовка данных

In [6]:
import re
import pymorphy2

Приведем буквы к нижнему регистру, оставим в тексте только русские буквы и проведем леммитизацию, для этого напишем функцию lemmatization

In [7]:
def lemmatization(data):
    reg = re.compile('[^абвгдеёжзийклмнопрстуфхцчшщъыьэюя ]')
    morph = pymorphy2.MorphAnalyzer()
    texts_clean = []
    for i in data:
        a = i.lower()
        b = reg.sub('', a)
        result = []
        for word in b.split():
            result.append(morph.parse(word)[0].normal_form)
            c = ' '.join(result)
        texts_clean.append(c)
    return (texts_clean)

In [8]:
texts_clean = lemmatization(texts)

# 3. Выбираем и обучаем модель на собранных отзывах

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score

C:\Users\Vera\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
cv = CountVectorizer(ngram_range=(2,4), min_df = 5,  analyzer='word')
X_train = cv.fit_transform(texts_clean)

Выберем модель с наилучшей предсказательной способностью

In [11]:
best_accuracy = 0
best_clf = 0
for clf in [LogisticRegression, LinearSVC, SGDClassifier, MultinomialNB]:
    model = clf()
    accuracy = cross_val_score(model, X_train, rating2, cv = 5).mean()
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_clf = clf
        
        
print(best_accuracy)
print(best_clf)

0.639125
<class 'sklearn.naive_bayes.MultinomialNB'>


C:\Users\Vera\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Vera\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Vera\Anaconda3\lib\site-p

По кросс - валидации лучшая модель - MultinomialNB.
Однако попробуем сдалать предсказание, используя Логистическую регрессию, но с балансировкой классов.

In [13]:
clf = LogisticRegression(class_weight='balanced', random_state = 2) 
clf.fit(X_train, rating2)
accuracy = cross_val_score(clf, X_train, rating2, cv = 5).mean()
accuracy

0.61587499999999995

Видим, что точность по кросс - валидации снизилась

Попробуем задать другой критерий деления на положительные и отрицательные отзывы. На этот раз к положительным отнесем отзывы с оценкой 4 и выше.

In [14]:
rating3 = []
for r in rating:
    if  r >= 4.0:
        rating3.append('pos')
    else:
        rating3.append('neg')
        
from collections import Counter
c = Counter(rating3)
c

Counter({'neg': 2144, 'pos': 5856})

Выборка стала еще более несбалансирована  
Построим модель и сделаем предсказание

In [15]:
clf = MultinomialNB() 
clf.fit(X_train, rating3)
accuracy = cross_val_score(clf, X_train, rating3, cv = 5).mean()
accuracy

0.72999402187266471

Видим что точность по кросс - валидации заметно повысилась с 0,64 до 0,73  
Подготовим тестовые данные и сделаем предсказание 

In [16]:
test_lemm = lemmatization(test_reviews_list)
X_test = cv.transform(test_lemm)

In [17]:
prediction = clf.predict(X_test)

In [18]:
df  = pd.DataFrame(prediction)
df.index.name = 'Id'
df.columns = ['y']
df.to_csv(r'D:\Python\Kaggle\Sent_an\submission4.csv', sep=',')

Точность на Kaggle получилась 0,76

Для улучшения предсказания попробуем еще поработать с выборкой:    
поскольку отзывы с оценкой 4, всегда давольно неопределенные, т.е. вроде телефон понравилься, но однако есть что то, что мещает поставить 5, и вполне возможно именно об этой детали и описано в отзыве. Поскольку у нас имеется достаточно большая выбрка, можем удалить отзывы с оценкой 4 и посмотеть как это повлияет на качество модели. К тому же это сделает выборку более сбалансированной


In [19]:
data['rating'].value_counts()

5.0    3905
4.0    1271
1.0     877
4.5     680
3.0     659
2.0     285
3.5     197
2.5      89
1.5      34
0.0       3
Name: rating, dtype: int64

In [20]:
data_clean = data[data['rating']!=4.0] 

То же может касаться и отзывов с оценкой 4.5, поэтому удалим и их из выборки

In [21]:
data_clean = data_clean[data_clean['rating']!=4.5]
texts = data_clean['text'].tolist()
rating = data_clean['rating'].tolist()

In [22]:
rating4 = []
for r in rating:
    if  r == 5.0:
        rating4.append('pos')
    else:
        rating4.append('neg')
        
from collections import Counter
c = Counter(rating4)
c

Counter({'neg': 2144, 'pos': 3905})

Видим, что количество положительных отзывов сократилось, хотя выборка осталась несбалансированной  
Применим нашу модель к новой выборке

In [24]:
texts_clean = lemmatization(texts)

In [25]:
cv = CountVectorizer(ngram_range=(2,4), min_df = 5,  analyzer='word')
X_train = cv.fit_transform(texts_clean)
clf = MultinomialNB() 
clf.fit(X_train, rating4)
accuracy = cross_val_score(clf, X_train, rating4, cv = 5).mean()
accuracy

0.71830158111683029

Качество по кросс - валидации немного снизилось.
Однако обобщающая спосбность модели, должна улучшиться. Поэтому сделаем предсказание и проверим

In [26]:
test_lemm = lemmatization(test_reviews_list)
X_test = cv.transform(test_lemm)
prediction = clf.predict(X_test)

In [27]:
df  = pd.DataFrame(prediction)
df.index.name = 'Id'
df.columns = ['y']
df.to_csv(r'D:\Python\Kaggle\Sent_an\submission.csv', sep=',')

Получили точность на Kaggle 0,80, это немного лучше. Поработаем еще с данными и вместо лемматизации будем использовать Cтемминг

In [28]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

Напишем функцию для чистки данных

In [29]:
def cleaning(data):
    reg = re.compile('[^абвгдеёжзийклмнопрстуфхцчшщъыьэюя ]')
    data_clean = []
    for i in data:
        a = i.lower()
        b = reg.sub('', a)
        data_clean.append(b)
    return(data_clean)

In [30]:
texts_clean_for_stem = cleaning(texts)

In [31]:
stemmer = PorterStemmer()
analyzer = CountVectorizer(ngram_range=(2,4), min_df=5, analyzer = 'word' ).build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))


stem_vectorizer = CountVectorizer(analyzer=stemmed_words)
vec_stem = stem_vectorizer.fit(texts_clean_for_stem)
X_train_stem = vec_stem.transform(texts_clean_for_stem)
X_train_stem

<6049x323552 sparse matrix of type '<class 'numpy.int64'>'
	with 375665 stored elements in Compressed Sparse Row format>

In [33]:
clf = MultinomialNB() 
clf.fit(X_train_stem, rating4)
accuracy = cross_val_score(clf, X_train, rating4, cv = 5).mean()
accuracy

0.71830158111683029

In [34]:
test_stem = cleaning(test_reviews_list)
X_test = vec_stem.transform(test_stem)
prediction = clf.predict(X_test)

In [36]:
df2 = pd.DataFrame(prediction)
df2.index.name = 'Id'
df2.columns = ['y']
df2.to_csv(r'D:\Python\Kaggle\Sent_an\submission2.csv', sep=',')

Качество на Kaggle улучшилось и составило 0,82, т.е. наша модель даст правильный ответ в 82% случаев, что довольно неплохо.

## Вывод: 
Для предсказания тональности отзыва были использованы 2 модели: LogisticRegression и MultinomialNB. Во всех случаях метод Байса показал наилучший результат. Результаты работы показывают, что подготовка и чистка данных является одним из наиболее важных этапов проведения анализа. Видно, что точность предсказания по кросс - валидации поднялась на ~10%, при изменении критерия разбиения отзывов на положительные и отрицательные. Точность предсказания на Kaggle поднялась на 5% после исключения отзывов которые с большой долей вероятности могут быть отнесены как к положительным, так и отрицательным. Еще на 2% удалось поднять точность при использовании стемминга вместо лемматизации. Можно и дальше повышать точность модели и добиться точности на Kaggle близкой к 100%, однако всегда надо помнить об опасности переобучения.